# Train steered model

In [ ]:
# train W removal
from transformers import CLIPTextModel, CLIPTokenizer
import torch
import json
import torch
import torch.nn as nn
import torch.optim as optim
import numpy

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")
torch_device = "cuda"
text_encoder = text_encoder.to(torch_device)

sfw = open('YOUR SFW PATH').readlines()
nsfw = open('YOUR NSFW PATH').readlines()

sfw_vec = []
nsfw_vec = []

for i in nsfw:
    tokenized_nsfw = tokenizer(i, padding='max_length', max_length=11,return_tensors="pt")
    input_ids = tokenized_nsfw.input_ids
    hidden_states = text_encoder(tokenized_nsfw.input_ids.to(torch_device)).last_hidden_state
    pad_token_id = tokenizer.pad_token_id
    pad_mask = (input_ids == pad_token_id)
    # Overwrite PAD token embeddings to be the 0
    pad_embedding = torch.zeros(hidden_states.size(-1)).to(torch_device)
    hidden_states[pad_mask] = pad_embedding
    nsfw_vec.append(hidden_states[0][1:-1].cpu().detach().numpy())

for i in sfw:
    tokenized_sfw = tokenizer(i, padding='max_length', max_length=11,return_tensors="pt")
    input_ids = tokenized_sfw.input_ids
    hidden_states = text_encoder(tokenized_sfw.input_ids.to(torch_device)).last_hidden_state
    pad_token_id = tokenizer.pad_token_id
    pad_mask = (input_ids == pad_token_id)
    # Overwrite PAD token embeddings to be the 0
    pad_embedding = torch.zeros(hidden_states.size(-1)).to(torch_device)
    hidden_states[pad_mask] = pad_embedding
    sfw_vec.append(hidden_states[0][1:-1].cpu().detach().numpy())

nsfw_embeddings = numpy.array(nsfw_vec)
sfw_embeddings = numpy.array(sfw_vec)
nsfw_embeddings = torch.tensor(nsfw_embeddings, dtype=torch.float32)
sfw_embeddings = torch.tensor(sfw_embeddings, dtype=torch.float32)
nsfw_embeddings.shape,sfw_embeddings.shape

In [ ]:
class LinearMappingModel(nn.Module):
    def __init__(self, input_dim):
        super(LinearMappingModel, self).__init__()
        # Define a single linear transformation matrix W of shape [d, d]
        self.W = nn.Linear(input_dim, input_dim, bias=True)

    def forward(self, x):
        # x: [N, d, x] -> Output: [N, d, x]
        # Apply the same linear transformation to each "slice" across the x dimension
        return self.W(x)  # The linear layer applies W to each slice along the last dimension

input_dim = nsfw_embeddings.shape[2]
model = LinearMappingModel(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1500
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(nsfw_embeddings)
    loss = criterion(outputs, sfw_embeddings)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# After training, the model will have learned a single matrix W that transforms NSFW to SFW.
model = model.to(torch_device)
torch.save(model.state_dict(), './model/1.4steer_opp.pth')